## Intro

- Kaggle Project: [House Prices - Advanced Regression Techniques](https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques/overview). Predict sales prices and practice feature engineering, RFs, and gradient boosting.
- Team: Yummy Donuts from [Data Donut](https://discord.gg/7fkzYbDxAh)
  - [Tina Yu](https://github.com/TinaHTYu)
  - [Zacks Shen](https://github.com/ZacksAmber)
- [Dataset Description](https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques/data)

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [2]:
train = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
# test = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')

---

## Metadata

In [3]:
# meta_data is from https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques/data
metadata = """
SalePrice - the property's sale price in dollars. This is the target variable that you're trying to predict.
MSSubClass: The building class
MSZoning: The general zoning classification
LotFrontage: Linear feet of street connected to property
LotArea: Lot size in square feet
Street: Type of road access
Alley: Type of alley access
LotShape: General shape of property
LandContour: Flatness of the property
Utilities: Type of utilities available
LotConfig: Lot configuration
LandSlope: Slope of property
Neighborhood: Physical locations within Ames city limits
Condition1: Proximity to main road or railroad
Condition2: Proximity to main road or railroad (if a second is present)
BldgType: Type of dwelling
HouseStyle: Style of dwelling
OverallQual: Overall material and finish quality
OverallCond: Overall condition rating
YearBuilt: Original construction date
YearRemodAdd: Remodel date
RoofStyle: Type of roof
RoofMatl: Roof material
Exterior1st: Exterior covering on house
Exterior2nd: Exterior covering on house (if more than one material)
MasVnrType: Masonry veneer type
MasVnrArea: Masonry veneer area in square feet
ExterQual: Exterior material quality
ExterCond: Present condition of the material on the exterior
Foundation: Type of foundation
BsmtQual: Height of the basement
BsmtCond: General condition of the basement
BsmtExposure: Walkout or garden level basement walls
BsmtFinType1: Quality of basement finished area
BsmtFinSF1: Type 1 finished square feet
BsmtFinType2: Quality of second finished area (if present)
BsmtFinSF2: Type 2 finished square feet
BsmtUnfSF: Unfinished square feet of basement area
TotalBsmtSF: Total square feet of basement area
Heating: Type of heating
HeatingQC: Heating quality and condition
CentralAir: Central air conditioning
Electrical: Electrical system
1stFlrSF: First Floor square feet
2ndFlrSF: Second floor square feet
LowQualFinSF: Low quality finished square feet (all floors)
GrLivArea: Above grade (ground) living area square feet
BsmtFullBath: Basement full bathrooms
BsmtHalfBath: Basement half bathrooms
FullBath: Full bathrooms above grade
HalfBath: Half baths above grade
Bedroom: Number of bedrooms above basement level
Kitchen: Number of kitchens
KitchenQual: Kitchen quality
TotRmsAbvGrd: Total rooms above grade (does not include bathrooms)
Functional: Home functionality rating
Fireplaces: Number of fireplaces
FireplaceQu: Fireplace quality
GarageType: Garage location
GarageYrBlt: Year garage was built
GarageFinish: Interior finish of the garage
GarageCars: Size of garage in car capacity
GarageArea: Size of garage in square feet
GarageQual: Garage quality
GarageCond: Garage condition
PavedDrive: Paved driveway
WoodDeckSF: Wood deck area in square feet
OpenPorchSF: Open porch area in square feet
EnclosedPorch: Enclosed porch area in square feet
3SsnPorch: Three season porch area in square feet
ScreenPorch: Screen porch area in square feet
PoolArea: Pool area in square feet
PoolQC: Pool quality
Fence: Fence quality
MiscFeature: Miscellaneous feature not covered in other categories
MiscVal: $Value of miscellaneous feature
MoSold: Month Sold
YrSold: Year Sold
SaleType: Type of sale
SaleCondition: Condition of sale
"""

In [4]:
def get_lines(string):
    cols, descs, data_types = [], [], []
    train_cols = train.columns
    
    # Split the string on the newline character to create a list of lines
    lines = string.split('\n')
    
    # Split each line into c and d for col and desc
    col, desc = lines[1].split(' - ')
    cols.append(col)
    descs.append(desc)
    data_type = str(train[col].dtype)
    data_types.append(data_type)
    # Split each line into c and d for col and desc
    for line in lines[2:]:  # Skip 0th: empty line; skip 1st: SalePrice - 
        if line != '':
            col, desc = line.split(': ')
            if col in train_cols:
                data_type = str(train[col].dtype)
            # Two cols don't have same names in train, Bedroom - BedroomAbvGr, Kitchen - KitchenAbvGr(52, 53)
            elif col == 'Bedroom':
                # We make the source data's col as the final col name
                col = 'BedroomAbvGr'
                data_type = str(train['BedroomAbvGr'].dtype)
            elif col == 'Kitchen':
                col = 'KitchenAbvGr'
                data_type = str(train['KitchenAbvGr'].dtype)
            cols.append(col)
            descs.append(desc)
            data_types.append(data_type)
    
    df = pd.DataFrame({'col': cols, 'desc': descs, 'source data type': data_types})
    
    return df

df = get_lines(metadata)
# Download this file from Kaggle notebook Output (see right panel),
# then copy and paste the file content to a Google sheet
# See https://docs.google.com/spreadsheets/d/1a9Xujf0RRGkbuMQxRL378L0dNUcwc2bz-GVxzeTmsT4
df.to_csv('meta_data.csv')